In [1]:
import pickle
with open('../intermediate/x.pickle','rb') as file:
    x = pickle.load(file)
with open('../intermediate/y.pickle','rb') as file:
    y = pickle.load(file)

In [2]:
import torch

# Python example
import sys
sys.path.append('../model/')  # Add the path to the 'lib' directory

import pp


In [3]:
import os

In [4]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def get_sentence_embeddings(sentences):
    # Tokenize the sentences
    encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Forward pass through the BERT model
    with torch.no_grad():
        outputs = model(**encoded_inputs)
        sentence_embeddings = outputs.last_hidden_state[:,0,:]  # Average pooling

    return sentence_embeddings

In [45]:
jgsvect = []
sumvect = []
similarity = []
i = 0
csvFolderPath = '../labelled_data/'
for filename in os.listdir(csvFolderPath):
    #access x[i] and sum i+1.txt

    #x[i]
    jv  = get_sentence_embeddings(x[i])
    i+=1
    jgsvect.append(jv)

    sumfile = filename[:-4] + '.txt'
    sumFolderpath = '../dataset/train-data/summary/'
    sumpath = os.path.join(sumFolderpath,sumfile)

    temp = []
    with open(sumpath,'r') as file:
        for line in file:
            temp.append(line.strip())
    sumlst = pp.preprocess(temp)
    sv = get_sentence_embeddings(sumlst)
    sumvect.append(sv)

    #calculate similarityfor each sentences in x[i] with its summary
    temp = []
    for jgsline in jv:
        simi = 0
        for sumline in sv:
            simi += (torch.nn.functional.cosine_similarity(sumline, jgsline, dim=0)).item()
        temp.append(simi)
    similarity.append(temp)

In [20]:
# for i in range(0,no_of_files):
#     dict = {1 : [], 2 : [], 3 : [], 4 : [], 5 : [], 6 : [], 7 : [], 8 : [], 9 : [], 10 : [], 11 : [], 12 : []}
#  #for each sentences in x[i]
#     for j in range(len(x[i])):
#         #x[i][j] is the jth sentence in ith judgement
#         # y[i][j] is a list containing labels
#         for label in y[i][j]:
#             if label!=0:
#                 dict[label].append([similarity[i][j],j])
    
#     #stored values in dict
#     #now for each dict[key] i will take last some k values and map rest other with 0
#     for key in dict:
#         lst = dict[key]
#         sorted_list = sorted(lst, key=lambda x: x[0], reverse=True)
#         sz = len(sorted_list)
#         if sz<=1:
#             continue
#         else:
#             #take first len/2 elements only i.e from 0 to len/2-1
#             for k in range(sz//2, sz):
#                 y[i][sorted_list[k][1]].remove(key)
#                 if len(y[i][sorted_list[k][1]])==0:
#                     y[i][sorted_list[k][1]].append(0)

In [46]:
import pickle
# Save data to a pickle file

with open('../intermediate/jgsvect.pickle', 'wb') as file:
    pickle.dump(jgsvect, file)

with open('../intermediate/sumvect.pickle', 'wb') as file:
    pickle.dump(sumvect, file)

with open('../intermediate/similarity.pickle', 'wb') as file:
    pickle.dump(similarity, file)


In [22]:
predjgs = '../dataset/test-data/judgement/1181.txt'
jgs = []
with open(predjgs,'r') as file:
    for line in file:
        jgs.append(line.strip())

In [24]:
jgslist = pp.preprocess(jgs)

In [27]:
temp = get_sentence_embeddings(jgslist)

In [29]:
with open('../intermediate/test.pickle', 'wb') as file:
    pickle.dump(temp, file)
with open('../intermediate/jgs.pickle', 'wb') as file:
    pickle.dump(jgs, file)